<a href="https://colab.research.google.com/github/Hiroyuki-01/lgbm_code/blob/main/0705_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

fold　８
rightgbm 
stratified k-fold


オリジナルで特徴量を追加した

### 1. 必要なライブラリをimport

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm
from lightgbm import LGBMClassifier

In [ ]:
!pip install optuna
import optuna

### 2. Colabでやっていたため、マウントしてます

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#cd drive/My Drive/GCI2020/Competition2/Model

### 3. 必要なCSVファイルを読み込み

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/gci2021_summer/competition_2/input/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/gci2021_summer/competition_2/input/test.csv')
#sample_submission = pd.read_csv('../sample_submission.csv')
df_all = pd.concat([train_df, test_df], axis=0)

In [ ]:
df_all.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,0.0,Cash loans,F,N,N,0,112500.0,755190.0,36328.5,675000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.010032,-9233,-878,-333.0,-522,NaN,1,1,1,1,0,0,Core staff,2.0,2,2,0,1,1,0,1,1,School,NaN,0.372591,NaN,0.0,0.0,0.0,0.0,-292.0,NaN,NaN,NaN,NaN
1,1,0.0,Cash loans,F,N,Y,0,225000.0,585000.0,16893.0,585000.0,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,0.008019,-20148,365243,-4469.0,-3436,NaN,1,0,0,1,0,0,NaN,2.0,2,2,0,0,0,0,0,0,XNA,NaN,0.449567,0.553165,0.0,0.0,0.0,0.0,-617.0,0.0,0.0,0.0,1.0
2,2,0.0,Cash loans,F,N,Y,0,54000.0,334152.0,18256.5,270000.0,Family,State servant,Secondary / secondary special,Married,House / apartment,0.004960,-18496,-523,-3640.0,-2050,NaN,1,1,1,1,1,0,Core staff,2.0,2,2,0,0,0,0,0,0,Postal,NaN,0.569503,NaN,4.0,0.0,4.0,0.0,-542.0,NaN,NaN,NaN,NaN
3,3,0.0,Cash loans,F,N,Y,0,67500.0,152820.0,8901.0,135000.0,Children,Pensioner,Lower secondary,Widow,House / apartment,0.005002,-24177,365243,-4950.0,-3951,NaN,1,0,0,1,1,0,NaN,1.0,3,3,0,0,0,0,0,0,XNA,NaN,0.105235,0.767523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1.0,Cash loans,M,N,N,0,157500.0,271066.5,21546.0,234000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,With parents,0.006296,-10685,-697,-5101.0,-3226,NaN,1,1,1,1,0,0,Drivers,2.0,3,3,0,0,0,0,1,1,Business Entity Type 3,0.342344,0.202490,0.669057,0.0,0.0,0.0,0.0,-1243.0,0.0,0.0,0.0,4.0


In [ ]:
train_df.shape

(171202, 51)

In [ ]:
test_df.shape

(61500, 50)

In [ ]:
df_all.shape

(232702, 51)

### 4. One Hot Encoding

In [ ]:
def one_hot_encoding(df):
    
    return_df = pd.get_dummies(df, drop_first=True)
    
    return return_df

### 5. 四則演算特徴量の作成

In [ ]:
def to_add_feature(df):
    df['OWN_CAR_AGE']=df['OWN_CAR_AGE'].fillna(0)
    df['EXT_SOURCE_1']=df['EXT_SOURCE_1'].fillna(df['EXT_SOURCE_1'].mean())
    df['EXTSOURCE_123_MEAN'] = (df['EXT_SOURCE_1'] + df['EXT_SOURCE_2'] + df['EXT_SOURCE_3']) / 3
    df['EXTSOURCE_23_MEAN'] = (df['EXT_SOURCE_2'] + df['EXT_SOURCE_3']) / 2
    df['EXTSOURCE_12_MEAN'] = (df['EXT_SOURCE_1'] + df['EXT_SOURCE_2']) / 2
    df['EXTSOURCE_13_MEAN'] = (df['EXT_SOURCE_1'] + df['EXT_SOURCE_3']) / 2
    df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    df['EXT_SOURCES_WEIGHTED'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 1 + df.EXT_SOURCE_3 * 3
    np.warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')
    for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
        feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
        df[feature_name] = eval('np.{}'.format(function_name))(
            df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    df['CREDIT_GOODS_PRICE'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['ANNUITY_INCOME_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
    df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']

    df['DAYS_BIRTH_365_OWN_CAR_AGE'] = (df['DAYS_BIRTH'] / 365) - df['OWN_CAR_AGE']
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    
    df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)
  
    return df
  

### 6. 不必要なカラムを消去

In [ ]:
def to_drop(df):
    
    drop_list = ['FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'REG_REGION_NOT_LIVE_REGION', 'LIVE_REGION_NOT_WORK_REGION']
    droped_df = df.drop(columns=drop_list)
    
    return droped_df

In [ ]:
df_encoded = one_hot_encoding(df_all)
added_features_df = to_add_feature(df_encoded)
all_features_df = to_drop(added_features_df)

### 7. お気持ちデバッグ

In [ ]:
assert len(df_all) == len(df_encoded)
assert len(df_all) == len(added_features_df)
assert len(df_all) == len(all_features_df)

In [ ]:
display(all_features_df)

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,...,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,EXTSOURCE_123_MEAN,EXTSOURCE_23_MEAN,EXTSOURCE_12_MEAN,EXTSOURCE_13_MEAN,EXT_SOURCES_PROD,EXT_SOURCES_WEIGHTED,EXT_SOURCES_MIN,EXT_SOURCES_MAX,EXT_SOURCES_MEAN,EXT_SOURCES_NANMEDIAN,EXT_SOURCES_VAR,PAYMENT_RATE,CREDIT_GOODS_PRICE,ANNUITY_INCOME_PERC,INCOME_TO_EMPLOYED_RATIO,INCOME_TO_BIRTH_RATIO,INCOME_PER_PERSON,DAYS_BIRTH_365_OWN_CAR_AGE,DAYS_EMPLOYED_PERC
0,0,0.0,0,112500.0,755190.0,36328.5,675000.0,0.010032,-9233,-878.0,-333.0,-522,0.0,1,1,0,0,2.0,2,2,1,0,1,1,0.498883,0.372591,NaN,0.0,0.0,0.0,0.0,-292.0,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0.435737,NaN,NaN,NaN,0.372591,0.498883,0.435737,0.435737,0.003987,0.048105,1.118800,0.148969,-128.132118,-12.184555,56250.0,-25.295890,0.095094
1,1,0.0,0,225000.0,585000.0,16893.0,585000.0,0.008019,-20148,NaN,-4469.0,-3436,0.0,0,0,0,0,2.0,2,2,0,0,0,0,0.498883,0.449567,0.553165,0.0,0.0,0.0,0.0,-617.0,0.0,0.0,0.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.500538,0.501366,0.474225,0.526024,0.124065,3.106827,0.449567,0.553165,0.500538,0.498883,0.001790,0.028877,1.000000,0.384615,0.616028,-11.167362,112500.0,-55.200000,NaN
2,2,0.0,0,54000.0,334152.0,18256.5,270000.0,0.004960,-18496,-523.0,-3640.0,-2050,0.0,1,1,1,0,2.0,2,2,0,0,0,0,0.498883,0.569503,NaN,4.0,0.0,4.0,0.0,-542.0,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0.534193,NaN,NaN,NaN,0.498883,0.569503,0.534193,0.534193,0.001247,0.054635,1.237600,0.161603,-103.250478,-2.919550,27000.0,-50.673973,0.028276
3,3,0.0,0,67500.0,152820.0,8901.0,135000.0,0.005002,-24177,NaN,-4950.0,-3951,0.0,0,0,1,0,1.0,3,3,0,0,0,0,0.498883,0.105235,0.767523,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.457214,0.436379,0.302059,0.633203,0.040295,3.405570,0.105235,0.767523,0.457214,0.498883,0.073973,0.058245,1.132000,0.441696,0.184808,-2.791910,67500.0,-66.238356,NaN
4,4,1.0,0,157500.0,271066.5,21546.0,234000.0,0.006296,-10685,-697.0,-5101.0,-3226,0.0,1,1,0,0,2.0,3,3,0,0,1,1,0.342344,0.202490,0.669057,0.0,0.0,0.0,0.0,-1243.0,0.0,0.0,0.0,4.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.404630,0.435773,0.272417,0.505700,0.046380,2.894348,0.202490,0.669057,0.404630,0.342344,0.038221,0.079486,1.158404,0.581038,-225.968436,-14.740290,78750.0,-29.273973,0.065232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
all_features_df.columns

Index(['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       ...
       'EXT_SOURCES_NANMEDIAN', 'EXT_SOURCES_VAR', 'PAYMENT_RATE',
       'CREDIT_GOODS_PRICE', 'ANNUITY_INCOME_PERC', 'INCOME_TO_EMPLOYED_RATIO',
       'INCOME_TO_BIRTH_RATIO', 'INCOME_PER_PERSON',
       'DAYS_BIRTH_365_OWN_CAR_AGE', 'DAYS_EMPLOYED_PERC'],
      dtype='object', length=161)

ext_scource_1 と　own_car_ageの欠損率が高い
targetも地味に高い→あたりまえ

In [ ]:
all_features_df.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.float).apply(lambda col: col/col.sum(), axis=0)

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,...,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,EXTSOURCE_123_MEAN,EXTSOURCE_23_MEAN,EXTSOURCE_12_MEAN,EXTSOURCE_13_MEAN,EXT_SOURCES_PROD,EXT_SOURCES_WEIGHTED,EXT_SOURCES_MIN,EXT_SOURCES_MAX,EXT_SOURCES_MEAN,EXT_SOURCES_NANMEDIAN,EXT_SOURCES_VAR,PAYMENT_RATE,CREDIT_GOODS_PRICE,ANNUITY_INCOME_PERC,INCOME_TO_EMPLOYED_RATIO,INCOME_TO_BIRTH_RATIO,INCOME_PER_PERSON,DAYS_BIRTH_365_OWN_CAR_AGE,DAYS_EMPLOYED_PERC
False,1.0,0.735713,1.0,1.0,1.0,0.999961,0.999072,1.0,1.0,0.821819,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.999991,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.997856,0.680811,0.996816,0.996816,0.996816,0.996816,0.876937,0.864079,0.864079,0.864079,0.864079,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.679534,0.679534,0.997856,0.680811,0.679534,0.679534,1.0,1.0,1.0,1.0,1.0,0.999961,0.999072,1.0,1.0,1.0,0.999991,1.0,0.821819
True,0.0,0.264287,0.0,0.0,0.0,0.000039,0.000928,0.0,0.0,0.178181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002144,0.319189,0.003184,0.003184,0.003184,0.003184,0.123063,0.135921,0.135921,0.135921,0.135921,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.320466,0.320466,0.002144,0.319189,0.320466,0.320466,0.0,0.0,0.0,0.0,0.0,0.000039,0.000928,0.0,0.0,0.0,0.000009,0.0,0.178181


### 8. データを分割

In [ ]:
train = all_features_df[all_features_df.loc[:, 'SK_ID_CURR'] < 171202]
test = all_features_df[all_features_df.loc[:, 'SK_ID_CURR'] > 171201]

NameError: ignored

train と　testに戻す。
testデータにはもともとtargetはないが、結合させたときにできたので削除する

In [ ]:
train_x = train.drop(columns=['TARGET', 'SK_ID_CURR'])
train_y = train['TARGET']
test_x = test.drop(columns=['TARGET', 'SK_ID_CURR'])

In [ ]:
train_x.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,...,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,EXTSOURCE_123_MEAN,EXTSOURCE_23_MEAN,EXTSOURCE_12_MEAN,EXTSOURCE_13_MEAN,EXT_SOURCES_PROD,EXT_SOURCES_WEIGHTED,EXT_SOURCES_MIN,EXT_SOURCES_MAX,EXT_SOURCES_MEAN,EXT_SOURCES_NANMEDIAN,EXT_SOURCES_VAR,PAYMENT_RATE,CREDIT_GOODS_PRICE,ANNUITY_INCOME_PERC,INCOME_TO_EMPLOYED_RATIO,INCOME_TO_BIRTH_RATIO,INCOME_PER_PERSON,DAYS_BIRTH_365_OWN_CAR_AGE,DAYS_EMPLOYED_PERC
0,0,112500.0,755190.0,36328.5,675000.0,0.010032,-9233,-878.0,-333.0,-522,0.0,1,1,0,0,2.0,2,2,1,0,1,1,0.498883,0.372591,NaN,0.0,0.0,0.0,0.0,-292.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0.435737,NaN,NaN,NaN,0.372591,0.498883,0.435737,0.435737,0.003987,0.048105,1.118800,0.148969,-128.132118,-12.184555,56250.0,-25.295890,0.095094
1,0,225000.0,585000.0,16893.0,585000.0,0.008019,-20148,NaN,-4469.0,-3436,0.0,0,0,0,0,2.0,2,2,0,0,0,0,0.498883,0.449567,0.553165,0.0,0.0,0.0,0.0,-617.0,0.0,0.0,0.0,1.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.500538,0.501366,0.474225,0.526024,0.124065,3.106827,0.449567,0.553165,0.500538,0.498883,0.001790,0.028877,1.000000,0.384615,0.616028,-11.167362,112500.0,-55.200000,NaN
2,0,54000.0,334152.0,18256.5,270000.0,0.004960,-18496,-523.0,-3640.0,-2050,0.0,1,1,1,0,2.0,2,2,0,0,0,0,0.498883,0.569503,NaN,4.0,0.0,4.0,0.0,-542.0,NaN,NaN,NaN,NaN,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0.534193,NaN,NaN,NaN,0.498883,0.569503,0.534193,0.534193,0.001247,0.054635,1.237600,0.161603,-103.250478,-2.919550,27000.0,-50.673973,0.028276
3,0,67500.0,152820.0,8901.0,135000.0,0.005002,-24177,NaN,-4950.0,-3951,0.0,0,0,1,0,1.0,3,3,0,0,0,0,0.498883,0.105235,0.767523,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.457214,0.436379,0.302059,0.633203,0.040295,3.405570,0.105235,0.767523,0.457214,0.498883,0.073973,0.058245,1.132000,0.441696,0.184808,-2.791910,67500.0,-66.238356,NaN
4,0,157500.0,271066.5,21546.0,234000.0,0.006296,-10685,-697.0,-5101.0,-3226,0.0,1,1,0,0,2.0,3,3,0,0,1,1,0.342344,0.202490,0.669057,0.0,0.0,0.0,0.0,-1243.0,0.0,0.0,0.0,4.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.404630,0.435773,0.272417,0.505700,0.046380,2.894348,0.202490,0.669057,0.404630,0.342344,0.038221,0.079486,1.158404,0.581038,-225.968436,-14.740290,78750.0,-29.273973,0.065232


辞書に含まれるすべての値を取得する

In [ ]:
X = train_x.values
y = train_y.values

optuna


In [ ]:
"""
from optuna.integration import lightgbm as lgb

In [ ]:
# LightGBM のデータセット表現にする
#lgb_train = lgb.Dataset(train_x, train_y)

In [ ]:
# データセットの分割方法
#folds = StratifiedKFold(n_splits=8, shuffle=True, random_state=69)

In [ ]:
"""
# 最適化するときの条件
lgbm_params = {
        'objective': 'binary',
        'metric': 'auc',
    }
# 基本的には cv() 関数のオプションがそのまま渡せる
tuner_cv = lgb.LightGBMTunerCV(
        lgbm_params, lgb_train,
        num_boost_round=1000,
        early_stopping_rounds=100,
        verbose_eval=20,
        folds=folds,
      )
    # 最適なパラメータを探索する
tuner_cv.run()

    # 最も良かったスコアとパラメータを書き出す
print(f'Best score: {tuner_cv.best_score}')
print('Best params:')
pprint(tuner_cv.best_params)
"""

optuna hold-out


> 




In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.25,random_state=0)

In [ ]:

def objective(trial):
    X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.25,random_state=42)


    #paramter_tuning using optuna
    bagging_freq =  trial.suggest_int('bagging_freq',1,10),
    min_data_in_leaf =  trial.suggest_int('min_data_in_leaf',2,100),
    max_depth = trial.suggest_int('max_depth',1,20),
    learning_rate = trial.suggest_loguniform('learning_rate',0.001,0.1),
    num_leaves = trial.suggest_int('num_leaves',2,70),
    num_threads = trial.suggest_int('num_threads',1,10),
    min_sum_hessian_in_leaf = trial.suggest_int('min_sum_hessian_in_leaf',1,10),
    
    model = LGBMClassifier(
        random_state = 0,
        verbosity = 1,
        bagging_seed = 0,
        boost_from_average = 'true',
        boost = 'gbdt',
        metric = 'auc',
        bagging_freq = bagging_freq ,
        min_data_in_leaf = min_data_in_leaf,
        max_depth = max_depth,
        learning_rate = learning_rate,
        num_leaves = num_leaves,
        num_threads = num_threads,
        min_sum_hessian_in_leaf = min_sum_hessian_in_leaf
    )

    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_valid)[:, 1]
    score = roc_auc_score(y_valid, y_pred)
    print(f'AUC: {score}')
    #show_hist(y_pred)
    #ightgbm.plot_importance(model,max_num_features=20)
    #model
    return 1-score

In [ ]:

# ハイパーパラメータの自動最適化
study = optuna.create_study()
study.optimize(objective,n_trials = 100)

In [ ]:
print('求めたハイパーパラメータ', study.best_params)

ValueError: ignored

In [ ]:
print('スコア　：', 1-study.best_value)

### 9. SteatifiedKFoldによるCVを準備

In [ ]:
fold = StratifiedKFold(n_splits=8, shuffle=True, random_state=69)
cv = list(fold.split(X, y))

In [ ]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 0.1,0.2),
    n_estimators, = trial.suggest_int('n_estimators', 20, 200),
    max_depth, = trial.suggest_int('max_depth', 3, 9),
    min_child_weight = trial.suggest_loguniform('min_child_weight', 0.5, 2),
    min_child_samples, = trial.suggest_int('min_child_samples', 5, 20),
    model = LGBMClassifier(learning_rate=learning_rate, 
                                    n_estimators=n_estimators,
                                    max_depth=max_depth, 
                                    min_child_weight=min_child_weight,
                                    min_child_samples=min_child_samples,
                                    subsample=0.8, colsample_bytree=0.8,
                                    verbose=-1, num_leaves=80)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_valid)[:, 1]
    score = roc_auc_score(y_valid, y_pred)
    #return classifier.score(X_train, y_train) # 正答率（train） の最適化
    return score # 尤度の最適化

In [ ]:
study = optuna.create_study(direction='maximize')

[I 2021-07-05 01:36:40,942] A new study created in memory with name: no-name-ba8468a0-0051-46d4-ab4a-22c0dc6c4edb


In [ ]:
study.optimize(objective, n_trials=100)

In [ ]:
study.best_params

{'colsample_bytree': 0.15589421725789662,
 'learning_rate': 2.221618231826747,
 'min_child_weight': 26.445486254677093,
 'reg_alpha': 13.695163190741468,
 'reg_lamda': 16.576495009439736,
 'subsample': 0.5730682533630741}

In [ ]:
study.best_value

0.7621099183433167

In [ ]:
params1=study.best_params

In [ ]:
params1

{'colsample_bytree': 0.15589421725789662,
 'learning_rate': 2.221618231826747,
 'min_child_weight': 26.445486254677093,
 'reg_alpha': 13.695163190741468,
 'reg_lamda': 16.576495009439736,
 'subsample': 0.5730682533630741}

### 10. Optunaを使ったハイパーパラメータの探索結果

In [ ]:
lgbm_best_param = {'reg_lambda': 1.1564659040946654, 'reg_alpha': 9.90877329623665, 'colsample_bytree': 0.5034991685866442, 'subsample': 0.6055998601661783, 'max_depth': 3, 'min_child_weight': 39.72586351155486, 'learning_rate': 0.08532489659779158}

optuna3

In [ ]:
def objective(trial):
    reg_lamda = trial.suggest_loguniform('reg_lamda', 0.01,10.0),
    reg_alpha = trial.suggest_loguniform('reg_alpha', 9.0, 20.0),
    colsample_bytree = trial.suggest_loguniform('colsample_bytree', 0.1, 1.0),
    min_child_weight = trial.suggest_loguniform('min_child_weight', 10.0, 40.0),
    subsample = trial.suggest_loguniform('subsample', 0.1, 3.0),
    learning_rate = trial.suggest_loguniform('learning_rate', 0.1, 3.0),
    model = LGBMClassifier(learning_rate=learning_rate, 
                                    n_estimators=176,
                                    max_depth=3, 
                                    min_child_weight=min_child_weight,
                                    subsample=subsample,
                                    reg_lamda=reg_lamda,
                                    reg_alpha=reg_alpha,
                                    colsample_bytree=colsample_bytree,
                                    verbose=-1,
                                    ramdom_state=71,
                           )
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_valid)[:, 1]
    score = roc_auc_score(y_valid, y_pred)
    #return classifier.score(X_train, y_train) # 正答率（train） の最適化
    return score # 尤度の最適化

In [ ]:
study = optuna.create_study(direction='maximize')

[I 2021-07-05 03:16:50,382] A new study created in memory with name: no-name-65e71860-5c12-4849-bda4-9366bff19b84


In [ ]:
study.optimize(objective, n_trials=100)

[W 2021-07-05 03:16:52,775] Trial 0 failed because of the following error: LightGBMError('Check failed: bagging_fraction <=1.0 at /__w/1/s/python-package/compile/src/io/config_auto.cpp, line 295 .\n')
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py", line 197, in train
    booster = Booster(params=params, train_set=train_set)
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py", line 1552, in __init__
    train_set.construct().handle,
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py", line 1001, in construct
    categorical_feature=self.categorical_feature, params=self.params)
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py", line 791, in _lazy_init
    self.__init_from_np2d(data, params_str, ref_dataset)
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py", line 855, in __init_from_np2d
    ctypes.byref(self.handle)))
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/b

LightGBMError: ignored

optuna4

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_validate

In [ ]:
def objectives(trial):
    # scikit-learnでお試しデータの準備


    # optunaでのハイパーパラメータサーチ範囲の設定
    params = {
        #fixed
        'boosting_type':'gbdt',
        'max_depth':-1,
        'learning_rate':0.1,
        'n_estimators': 1000,
        'metric':'l2',

        #variable
        'num_leaves': trial.suggest_int('num_leaves', 10, 300),
        'reg_alpha': trial.suggest_loguniform('reg_alpha',0.001, 10),
        'reg_lambda':trial.suggest_loguniform('reg_lambda', 0.001, 10),

    }

    # LightGBMで学習+予測
    model = LGBMClassifier(**params,random_state=0)# 追加部分
    
    # kFold交差検定で決定係数を算出し、各セットの平均値を返す
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    scores = cross_validate(model, X=X_train, y=y_train,scoring='roc_auc',cv=kf)
    return scores

In [ ]:
opt = optuna.create_study(direction='maximize')
opt.optimize(objectives, n_trials=5)

[I 2021-07-05 02:37:22,842] A new study created in memory with name: no-name-3b4397ec-ec7b-43f5-9a0f-f222a9df4054


KeyboardInterrupt: ignored

### 11. CVするための関数作成

In [ ]:
def fit_lgbm(X, y, cv, params: dict=None, verbose=100):
    
    oof_preds = np.zeros(X.shape[0])

    if params is None:
        params = {}

    models = []

    for i, (idx_train, idx_valid) in enumerate(cv):
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMClassifier(**params, random_state=71, n_estimators=10000)
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100, 
                eval_metric='auc',
                verbose=verbose)

        models.append(clf)
        oof_preds[idx_valid] = clf.predict_proba(x_valid, num_iteration=clf.best_iteration_)[:, 1]
        print('Fold %2d AUC : %.6f' % (i + 1, roc_auc_score(y_valid, oof_preds[idx_valid])))
    
    score = roc_auc_score(y, oof_preds)
    print('Full AUC score %.6f' % score) 
    return oof_preds, models

In [ ]:
oof, models = fit_lgbm(X, y, cv=cv, params=lgbm_best_param)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.750457	valid_0's binary_logloss: 0.248649
[200]	valid_0's auc: 0.754866	valid_0's binary_logloss: 0.24726
[300]	valid_0's auc: 0.757272	valid_0's binary_logloss: 0.246484
[400]	valid_0's auc: 0.75907	valid_0's binary_logloss: 0.245945
[500]	valid_0's auc: 0.76025	valid_0's binary_logloss: 0.245731
[600]	valid_0's auc: 0.760994	valid_0's binary_logloss: 0.24557
[700]	valid_0's auc: 0.761294	valid_0's binary_logloss: 0.245528
[800]	valid_0's auc: 0.761939	valid_0's binary_logloss: 0.245413
Early stopping, best iteration is:
[785]	valid_0's auc: 0.76201	valid_0's binary_logloss: 0.245359
Fold  1 AUC : 0.762010
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.753793	valid_0's binary_logloss: 0.247456
[200]	valid_0's auc: 0.759124	valid_0's binary_logloss: 0.245592
[300]	valid_0's auc: 0.761854	valid_0's binary_logloss: 0.244768
[400]	valid_0's auc: 0.763309	valid_0's b

0.763402
0.763724

In [ ]:
oof

array([0.11885829, 0.04829463, 0.10867399, ..., 0.01500345, 0.04248297,
       0.08407765])

### 12. 予測

In [ ]:
pred = np.array([model.predict_proba(test_x.values)[:, 1] for model in models])

In [ ]:
pred

array([[0.02464405, 0.18414692, 0.21499201, ..., 0.04664491, 0.20682515,
        0.0991185 ],
       [0.02736211, 0.21020111, 0.17729741, ..., 0.06017648, 0.19855474,
        0.11205077],
       [0.01461316, 0.17581487, 0.20109193, ..., 0.07123544, 0.20276647,
        0.13163518],
       ...,
       [0.02223938, 0.18122149, 0.16723239, ..., 0.06493153, 0.14456494,
        0.13151281],
       [0.02703577, 0.19822806, 0.20389576, ..., 0.07008591, 0.23377687,
        0.11293584],
       [0.02446726, 0.16700108, 0.21821282, ..., 0.05683348, 0.20029513,
        0.15155832]])

In [ ]:
pred = np.mean(pred, axis=0)

In [ ]:
pred

array([0.02338136, 0.18827721, 0.19654425, ..., 0.06033082, 0.19827683,
       0.12718805])

In [ ]:
submission_path='/content/drive/MyDrive/gci2021_summer/competition_2/sample_submission.csv'
submission = pd.read_csv(submission_path)

In [ ]:
submission

,SK_ID_CURR,TARGET
0,171202,0.5
1,171203,0.5
2,171204,0.5
3,171205,0.5
4,171206,0.5
...,...,...
61495,232697,0.5
61496,232698,0.5
61497,232699,0.5
61498,232700,0.5


In [ ]:
#submission = sample_submission.copy()
submission['TARGET'] = pred

In [ ]:
# submission.to_csv('../submissions/3rd_place_solution.csv', index=False)

In [ ]:
submission

,SK_ID_CURR,TARGET
0,171202,0.023381
1,171203,0.188277
2,171204,0.196544
3,171205,0.097935
4,171206,0.168279
...,...,...
61495,232697,0.157101
61496,232698,0.026157
61497,232699,0.060331
61498,232700,0.198277


In [ ]:
submission_path='/content/drive/MyDrive/Colab Notebooks/competition_2_note/0705-optuna.csv'
submission.to_csv(submission_path)